In [1]:
import numpy as np
import pandas as pd
import os
import healpy as hp
from DS_data_transformation import get_prm
%config InlineBackend.figure_format = 'retina'
from DS_Planck_Unet import west_val, east_val, east
from matplotlib import pyplot as plt
from DS_Planck_Unet import load_planck_model, draw_pic_with_mask, draw_pic, val_pix
import os
from tqdm.notebook import tqdm
from DS_models_stat import stat_orig_cats, do_all_stats, stat_cat, cut_cat
from DS_detector import rematch_cat
from DS_healpix_fragmentation import cut_cat_by_pix

In [2]:
max_pred_unite = np.arange(0.1, 1, 0.05)
max_pred_br = np.array(pd.read_csv('./check/old/brcat_recall_max_pred_pix41.csv')['s/n'])

In [3]:
def cut_max_pred(cat, max_pred):
    df = cat.copy()
    df = df[df['max_pred'] >= max_pred]
    df.index = np.arange(len(df))
    return df

In [4]:
pixels = {'41' : [41], '38' : [38], '6' : [6], 'e' : east_val, 'w' : west_val}

In [5]:
s = '/home/rt2122/Data/gen_cats/gen_all_found_brcat_prec0.2.csv'
def get_prec(s):
    return float(s[s.find('prec')+4:-4])
get_prec(s)

0.2

In [20]:
def get_cat_d(det_name, det_file, get_mpp_min=False):
    cat_d = {}
    if det_name == 'gen':
        dirname = det_file 
        files = next(os.walk(dirname))[-1]
        cat_d = {get_prec(file) : pd.read_csv(os.path.join(dirname, file)) for file in files}
    else:
        det_cat = pd.read_csv(det_file)
        max_pred_vals = max_pred_unite
        if det_name in ['SZcat(y-map)', 'SZcat(y-map)_unique', 'inter_sn']:
            max_pred_vals = max_pred_br
        cat_d = {max_pred_prm : cut_max_pred(det_cat, max_pred_prm) for max_pred_prm in max_pred_vals}
        if get_mpp_min:
            new_cat_d = {}
            for val in cat_d:
                df = cat_d[val]
                new_cat_d[df['mpp'].min()] = df
            cat_d = new_cat_d
    return cat_d

In [21]:
def get_recalls_file(det_name, det_file, big_pix, pixname, spec_precision=['PSZ2(z)', 'eROSITA'], index_name='max_pred',
                    dict_cut={}):
    cat_d = get_cat_d(det_name, det_file)
    recall_df = stat_cat(cat_d, big_pix=big_pix, read_det_files=False, match_dist=400/3600, 
                                     spec_precision=spec_precision, dict_cut=dict_cut)
    recall_df = recall_df.sort_index()
    recall_df.index.name = index_name
    filename = './check/{}_recall_{}_pix{}.csv'.format(det_name, index_name.replace('/', ''), pixname)
    recall_df.to_csv(filename)

In [19]:
cats_dict_max = {
            'SZcat(base)' : '/home/rt2122/Data/detected_cats/full_pz_rot28_thr0.1_step8_no_fn.csv',
            'SZcat(AL)' : '/home/rt2122/Data/detected_cats/full_pz_all_found34_thr0.1_step8_no_fn.csv',
            'SZcat(AL)_unique' : '/home/rt2122/Data/inter_cats/all_found_unique.csv',
            'inter' : '/home/rt2122/Data/inter_cats/inter.csv',
}
cats_dict_sn = {
            'SZcat(y-map)' : '/home/rt2122/Data/SRGz/brcat.csv',
            'inter_sn' : '/home/rt2122/Data/inter_cats/inter_sn.csv',
            'SZcat(y-map)_unique': '/home/rt2122/Data/inter_cats/brcat_unique.csv'
}
cats_dict_mpp = {
            'SZcat(base)' : '/home/rt2122/Data/detected_cats/full_pz_rot28_thr0.1_step8_no_fn_mpp.csv',
            'SZcat(AL)' : '/home/rt2122/Data/detected_cats/full_pz_all_found34_thr0.1_step8_no_fn_mpp.csv',
            'SZcat(y-map)' : '/home/rt2122/Data/SRGz/brcat_mpp.csv',
    
            'gen' : '/home/rt2122/Data/gen_cats/',
    
            'SZcat(AL)_unique' : '/home/rt2122/Data/inter_cats/all_found_unique_mpp.csv',
            'inter' : '/home/rt2122/Data/inter_cats/inter_mpp.csv',
            'SZcat(y-map)_unique': '/home/rt2122/Data/inter_cats/brcat_unique_mpp.csv'
}

In [22]:
name = 'SZcat(AL)'
dict_by_index = zip([cats_dict_max, cats_dict_sn, cats_dict_mpp], ['max_pred', 's/n', 'mpp'])
for dict_cur, index_name in dict_by_index:
    for name in tqdm(dict_cur):
        get_recalls_file(name, dict_cur[name], list(range(48)), pixname='eb20',
                                   dict_cut={'b' : [20, np.inf], 'l' : [0, 180]})
        for pixname in pixels:
            get_recalls_file(name, dict_cur[name], big_pix=pixels[pixname], pixname=pixname, dict_cut={})

/home/rt2122/L/git/data-segmentation-2/modules/DS_models_stat.py:169: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  det_cat['found'] = False
/home/rt2122/L/git/data-segmentation-2/modules/DS_models_stat.py:171: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  det_cat['found_' + cat] = False
/home/rt2122/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value ins

In [24]:
cats_dict = {'SZcat(base)' : '/home/rt2122/Data/detected_cats/full_pz_rot28_thr0.1_step8_no_fn.csv',
            'SZcat(AL)' : '/home/rt2122/Data/detected_cats/full_pz_all_found34_thr0.1_step8_no_fn.csv',
            'SZcat(y-map)' : '/home/rt2122/Data/SRGz/brcat.csv',
            'gen' : '/home/rt2122/Data/gen_cats/',
            'SZcat(AL)_unique' : '/home/rt2122/Data/inter_cats/all_found_unique.csv',
            'inter' : '/home/rt2122/Data/inter_cats/inter.csv',
            'inter_sn' : '/home/rt2122/Data/inter_cats/inter_sn.csv',
            'SZcat(y-map)_unique': '/home/rt2122/Data/inter_cats/brcat_unique.csv'}
for pixname in tqdm(pixels):
    big_pix = pixels[pixname] 
    for det_name in cats_dict:
        det_cat = None
        cat_d = get_cat_d(det_name, cats_dict[det_name])

        recall_df = stat_cat(cat_d, big_pix=big_pix, read_det_files=False, match_dist=400/3600, 
                                         spec_precision=['PSZ2(z)', 'eROSITA'])
        recall_df = recall_df.sort_index()
        recall_df.index.name='max_pred'
        if det_name in ['SZcat(y-map)', 'inter_sn', 'SZcat(y-map)_unique']:
            recall_df.index.name='s/n'
        if det_name in ['gen']:
            recall_df.index.name='mpp'
        filename = './check/{}_recall_{}_pix{}.csv'.format(det_name, str(recall_df.index.name).replace('/', ''), pixname)
        recall_df.to_csv(filename)
        print(filename)

SZcat(base) [0.1  0.15 0.2  0.25 0.3  0.35 0.4  0.45 0.5  0.55 0.6  0.65 0.7  0.75
 0.8  0.85 0.9  0.95]
./check/SZcat(base)_recall_max_pred_pix41.csv
SZcat(AL) [0.1  0.15 0.2  0.25 0.3  0.35 0.4  0.45 0.5  0.55 0.6  0.65 0.7  0.75
 0.8  0.85 0.9  0.95]
./check/SZcat(AL)_recall_max_pred_pix41.csv
SZcat(y-map) [ 5.05335105  5.1069375   5.1630591   5.223298    5.29437     5.3639328
  5.43414885  5.5157134   5.60974235  5.711204    5.8320221   5.9664762
  6.12062285  6.2958075   6.52273275  6.826352    7.2506403   8.0152959
 10.0664315 ]


/home/rt2122/L/git/data-segmentation-2/modules/DS_models_stat.py:169: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  det_cat['found'] = False
/home/rt2122/L/git/data-segmentation-2/modules/DS_models_stat.py:171: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  det_cat['found_' + cat] = False
/home/rt2122/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value ins

./check/SZcat(y-map)_recall_sn_pix41.csv
./check/gen_recall_mpp_pix41.csv
SZcat(AL)_unique [0.1  0.15 0.2  0.25 0.3  0.35 0.4  0.45 0.5  0.55 0.6  0.65 0.7  0.75
 0.8  0.85 0.9  0.95]
./check/SZcat(AL)_unique_recall_max_pred_pix41.csv
inter [0.1  0.15 0.2  0.25 0.3  0.35 0.4  0.45 0.5  0.55 0.6  0.65 0.7  0.75
 0.8  0.85 0.9  0.95]
./check/inter_recall_max_pred_pix41.csv
inter_sn [ 5.05335105  5.1069375   5.1630591   5.223298    5.29437     5.3639328
  5.43414885  5.5157134   5.60974235  5.711204    5.8320221   5.9664762
  6.12062285  6.2958075   6.52273275  6.826352    7.2506403   8.0152959
 10.0664315 ]
./check/inter_sn_recall_sn_pix41.csv
SZcat(y-map)_unique [ 5.05335105  5.1069375   5.1630591   5.223298    5.29437     5.3639328
  5.43414885  5.5157134   5.60974235  5.711204    5.8320221   5.9664762
  6.12062285  6.2958075   6.52273275  6.826352    7.2506403   8.0152959
 10.0664315 ]
./check/SZcat(y-map)_unique_recall_sn_pix41.csv
SZcat(base) [0.1  0.15 0.2  0.25 0.3  0.35 0.4  0.45

In [31]:
big_pix = list(range(48))
pixname='eb20'
cats_dict = {'SZcat(base)' : '/home/rt2122/Data/detected_cats/full_pz_rot28_thr0.1_step8_no_fn.csv',
            'SZcat(AL)' : '/home/rt2122/Data/detected_cats/full_pz_all_found34_thr0.1_step8_no_fn.csv',
            'SZcat(y-map)' : '/home/rt2122/Data/SRGz/brcat.csv',
            'gen' : '/home/rt2122/Data/gen_cats/',
            'SZcat(AL)_unique' : '/home/rt2122/Data/inter_cats/all_found_unique.csv',
            'inter' : '/home/rt2122/Data/inter_cats/inter.csv',
            'inter_sn' : '/home/rt2122/Data/inter_cats/inter_sn.csv',
            'SZcat(y-map)_unique': '/home/rt2122/Data/inter_cats/brcat_unique.csv'}
for det_name in tqdm(cats_dict):
    cat_d = get_cat_d(det_name, cats_dict[det_name])
    for val in cat_d:
        df = cat_d[val]
        #df = cut_cat(df, dict_cut={'b' : [20, np.inf]})
        cat_d[val] = df

    recall_df = stat_cat(cat_d, big_pix=big_pix, read_det_files=False, match_dist=400/3600, 
                                     spec_precision=['PSZ2(z)', 'eROSITA'], dict_cut={'b' : [20, np.inf], 'l' : [0, 180]})
    recall_df = recall_df.sort_index()
    recall_df.index.name='max_pred'
    if det_name in ['SZcat(y-map)', 'inter_sn', 'SZcat(y-map)_unique']:
        recall_df.index.name='s/n'
    if det_name in ['gen']:
        recall_df.index.name='mpp'
    filename='./check/{}_recall_{}_pix{}.csv'.format(det_name, recall_df.index.name.replace('/', ''), pixname)
    recall_df.to_csv(filename)
    print(filename)
    print(recall_df.index)

cats_dict = {'SZcat(base)' : '/home/rt2122/Data/detected_cats/full_pz_rot28_thr0.1_step8_no_fn_mpp.csv',
            'SZcat(AL)' : '/home/rt2122/Data/detected_cats/full_pz_all_found34_thr0.1_step8_no_fn_mpp.csv',
            'SZcat(y-map)' : '/home/rt2122/Data/SRGz/brcat_mpp.csv',
            'gen' : '/home/rt2122/Data/gen_cats/',
            'SZcat(AL)_unique' : '/home/rt2122/Data/inter_cats/all_found_unique_mpp.csv',
            'inter' : '/home/rt2122/Data/inter_cats/inter_mpp.csv',
            'SZcat(y-map)_unique': '/home/rt2122/Data/inter_cats/brcat_unique_mpp.csv'}
for det_name in tqdm(cats_dict):
    cat_d = get_cat_d(det_name, cats_dict[det_name], get_mpp_min=True)
    for val in cat_d:
        df = cat_d[val]
        df = cut_cat(df, dict_cut={'b' : [20, np.inf]})
        cat_d[val] = df

    recall_df = stat_cat(cat_d, big_pix=big_pix, read_det_files=False, match_dist=400/3600, 
                                     spec_precision=['PSZ2(z)', 'eROSITA'])
    recall_df = recall_df.sort_index()
    recall_df.index.name='max_pred'
    if det_name in ['SZcat(y-map)', 'inter_sn', 'SZcat(y-map)_unique']:
        recall_df.index.name='s/n'
    if det_name in ['gen']:
        recall_df.index.name='mpp'
    filename = './check/{}_recall_{}_pix{}.csv'.format(det_name, recall_df.index.name.replace('/', ''), pixname)
    recall_df.to_csv(filename)
    print(filename)

SZcat(base) [0.1  0.15 0.2  0.25 0.3  0.35 0.4  0.45 0.5  0.55 0.6  0.65 0.7  0.75
 0.8  0.85 0.9  0.95]
./check/SZcat(base)_recall_max_pred_pixeb20.csv
Float64Index([                0.1, 0.15000000000000002, 0.20000000000000004,
              0.25000000000000006, 0.30000000000000004,  0.3500000000000001,
              0.40000000000000013, 0.45000000000000007,  0.5000000000000001,
               0.5500000000000002,  0.6000000000000002,  0.6500000000000001,
               0.7000000000000002,  0.7500000000000002,  0.8000000000000002,
               0.8500000000000002,  0.9000000000000002,  0.9500000000000003],
             dtype='float64', name='max_pred')
SZcat(AL) [0.1  0.15 0.2  0.25 0.3  0.35 0.4  0.45 0.5  0.55 0.6  0.65 0.7  0.75
 0.8  0.85 0.9  0.95]
./check/SZcat(AL)_recall_max_pred_pixeb20.csv
Float64Index([                0.1, 0.15000000000000002, 0.20000000000000004,
              0.25000000000000006, 0.30000000000000004,  0.3500000000000001,
              0.40000000000000013,

SZcat(base) [0.1  0.15 0.2  0.25 0.3  0.35 0.4  0.45 0.5  0.55 0.6  0.65 0.7  0.75
 0.8  0.85 0.9  0.95]
./check/SZcat(base)_recall_max_pred_pixeb20.csv
SZcat(AL) [0.1  0.15 0.2  0.25 0.3  0.35 0.4  0.45 0.5  0.55 0.6  0.65 0.7  0.75
 0.8  0.85 0.9  0.95]
./check/SZcat(AL)_recall_max_pred_pixeb20.csv
SZcat(y-map) [ 5.05335105  5.1069375   5.1630591   5.223298    5.29437     5.3639328
  5.43414885  5.5157134   5.60974235  5.711204    5.8320221   5.9664762
  6.12062285  6.2958075   6.52273275  6.826352    7.2506403   8.0152959
 10.0664315 ]
./check/SZcat(y-map)_recall_sn_pixeb20.csv
./check/gen_recall_mpp_pixeb20.csv
SZcat(AL)_unique [0.1  0.15 0.2  0.25 0.3  0.35 0.4  0.45 0.5  0.55 0.6  0.65 0.7  0.75
 0.8  0.85 0.9  0.95]
./check/SZcat(AL)_unique_recall_max_pred_pixeb20.csv
inter [0.1  0.15 0.2  0.25 0.3  0.35 0.4  0.45 0.5  0.55 0.6  0.65 0.7  0.75
 0.8  0.85 0.9  0.95]
./check/inter_recall_max_pred_pixeb20.csv
SZcat(y-map)_unique [ 5.05335105  5.1069375   5.1630591   5.223298    5.2

In [19]:
cats_dict = {'SZcat(base)' : '/home/rt2122/Data/detected_cats/full_pz_rot28_thr0.1_step8_no_fn_mpp.csv',
            'SZcat(AL)' : '/home/rt2122/Data/detected_cats/full_pz_all_found34_thr0.1_step8_no_fn_mpp.csv',
            'SZcat(y-map)' : '/home/rt2122/Data/SRGz/brcat_mpp.csv',
            'gen' : '/home/rt2122/Data/gen_cats/',
            'SZcat(AL)_unique' : '/home/rt2122/Data/inter_cats/all_found_unique_mpp.csv',
            'inter' : '/home/rt2122/Data/inter_cats/inter_mpp.csv',
            'SZcat(y-map)_unique': '/home/rt2122/Data/inter_cats/brcat_unique_mpp.csv'}
for pixname in tqdm(pixels):
    big_pix = pixels[pixname] 
    for det_name in cats_dict:
        cat_d = get_cat_d(det_name, cats_dict[det_name], get_mpp_min=det_name!='gen')

        recall_df = stat_cat(cat_d, big_pix=big_pix, read_det_files=False, match_dist=400/3600, 
                                         spec_precision=['PSZ2(z)', 'eROSITA'])
        recall_df = recall_df.sort_index()
        recall_df.index.name='mpp'
        recall_df.to_csv('./check/{}_recall_mpp_pix{}.csv'.format(det_name, pixname))

In [10]:
list(cats_dict.keys())

['SZcat(base)',
 'SZcat(AL)',
 'brcat',
 'gen',
 'SZcat(AL)_unique',
 'inter',
 'brcat_unique']

In [35]:
df = pd.read_csv('./check/SZcat(base)_recall_max_pred_pixeb20.csv')

In [36]:
df

,max_pred,PSZ2,MCXC,ACT,eROSITA,precision_eROSITA,found_eROSITA,PSZ2(z),precision_PSZ2(z),found_PSZ2(z),all_true,found,precision,all
0,0.329840,0.765275,0.420539,0.241955,0.103844,0.119512,1900,0.867459,0.059567,947,0.236054,3323,0.209020,15898
1,0.364932,0.756201,0.406770,0.222646,0.092114,0.135676,1676,0.864717,0.076419,944,0.219470,2986,0.241723,12353
2,0.394682,0.750756,0.398164,0.210012,0.084672,0.148658,1534,0.862888,0.091288,942,0.208271,2774,0.268824,10319
3,0.428655,0.744707,0.388411,0.199046,0.078561,0.157555,1415,0.861060,0.104665,940,0.199332,2595,0.288943,8981
4,0.454022,0.741077,0.375215,0.188081,0.073879,0.167361,1326,0.859232,0.118389,938,0.190610,2460,0.310488,7923
5,0.482090,0.735027,0.367183,0.181883,0.069887,0.179293,1252,0.857404,0.134040,936,0.184687,2339,0.334956,6983
6,0.525088,0.727768,0.359725,0.173302,0.066042,0.190147,1181,0.853748,0.150056,932,0.177256,2217,0.356947,6211
7,0.543346,0.719903,0.355709,0.164482,0.063282,0.204381,1129,0.849177,0.167813,927,0.170902,2120,0.383780,5524
8,0.568293,0.712644,0.352266,0.157092,0.059980,0.216922,1064,0.845521,0.188175,923,0.164549,2014,0.410601,4905
9,0.595463,0.699335,0.345955,0.149702,0.057171,0.231403,1011,0.839122,0.209659,916,0.158949,1919,0.439231,4369
